In [ ]:
import numpy as np
from geopy.distance import geodesic

import folium
import matplotlib.pyplot as plt

from amplify import sum_poly, Solver, BinarySymbolGenerator
from amplify.constraint import less_equal, one_hot
from amplify.client import FixstarsClient

from collections import namedtuple
from IPython.display import display
from ipywidgets import (
    Button,
    FloatSlider,
    IntSlider,
    interactive_output,
    VBox,
    HBox,
    Output,
    Label,
    Accordion,
    IntProgress,
    GridBox,
    HTML,
    GridspecLayout,
)

# 船橋駅周辺
lon = (139.9, 140.08)
lat = (35.675500, 35.76)
# 9箇所
parking = [
    (35.67699938102926, 140.0434199237448),
    (35.68494726920934, 139.99303731029542),
    (35.68604762650153, 140.01831984588475),
    (35.69720660219214, 139.98034538800417),
    (35.6981824540223, 140.00360550271415),
    (35.698774929464875, 139.9982410856558),
    (35.700029569368, 139.98558105961536),
    (35.70599837320516, 139.93269833544272),
    (35.71199204224218, 140.0415316476293),
]
_colors = [
    "green",
    "orange",
    "blue",
    "red",
    "purple",
    "pink",
    "darkblue",
    "cadetblue",
    "darkred",
    "lightred",
    "darkgreen",
    "lightgreen",
    "lightblue",
    "darkpurple",
]
zoom = 10.5

# Ride sharing

今回取り扱う問題は、集合型ライドシェアと呼ばれる問題です。  
集合型ライドシェアとは、複数の利用者がいくつかの大型駐車場に集合し、
同じ車に乗って同じ目的地を目指す形式のライドシェアを指します。


次のデモでは、同じ目的地を持つ複数の人物と利用可能な車の候補が与えられている場合に、各人の駐車場までの移動距離と使用する車の台数をなるべく小さくするような人と車の割り当てを求めます。イジングモデルとして定式化を行い、Amplify Annealing Engineを用いて最小化問題として割り当てを求めていきます。  
「Run」ボタンを押すと実行されます。  
「Change Location」ボタンを押すと利用者の配置を変更することができます。  
「Options」から問題設定や実行時のパラメーター等の変更が可能です。

In [ ]:
def generate_problem(
    lon_range,
    lat_range,
    parking,
    ncars=None,
    npeople=None,
    C=None,
    lb=1,
    ub=160,
    seed=1,
):
    """
    車の数、人の数、車の定員をランダムに決定した後、
    車の数＋人の数の点の座標を生成し、座標を元に距離行列を生成する関数
    """
    np.random.seed(seed)
    if ncars is None or (isinstance(ncars, int) and ncars > len(parking)):
        if isinstance(ncars, int) and ncars > len(parking):
            print(
                f"Maximum value of ncars is {len(parking)}.\n ncars : {ncars} -> {len(parking)}."
            )
        ncars = len(parking)
    if npeople is None:
        npeople = np.random.randint(lb, ub)
    if C is None:
        C = np.random.randint(npeople // ncars + 1, npeople + 2)
    if ncars * C < npeople:
        display(
            HTML(
                (
                    "<h3><font color='red'>利用者数が乗車可能人数を超えています。パラメーターを変更してください。</font></h3>"
                )
            )
        )
        raise RuntimeError
    n = ncars + npeople
    ind2coord = dict()
    tmp = [
        parking[i][::-1] for i in np.random.choice(len(parking), ncars, replace=False)
    ]
    for i in range(ncars):
        ind2coord[i] = (tmp[i][0], tmp[i][1])
    for i in range(ncars, n):
        lon = np.random.uniform(lon_range[0], lon_range[1])
        lat = np.random.uniform(lat_range[0], lat_range[1])
        tmp.append((lon, lat))
        ind2coord[i] = (lon, lat)

    D = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            D[i, j] = geodesic(tmp[i][::-1], tmp[j][::-1]).m
    return ncars, npeople, D, C, ind2coord

In [ ]:
def regularizeDistance(D):
    average = D.mean(axis=0, keepdims=True)
    std = D.std(axis=0, keepdims=True, ddof=0)
    return (D - average) / std

In [ ]:
def setObjective(q, ncars, npeople, D, C, alpha=1):
    """目的関数"""
    # 各利用者の移動距離に関係する項
    distance_cost = sum_poly(D[ncars:, :ncars] * q)

    # 各車の乗車率に関係する項
    ride_rate_cost = ((q.sum(axis=0) / C) ** 2).sum()

    cost = distance_cost - alpha * ride_rate_cost
    return cost

In [ ]:
def setConstraints(q, ncars, npeople, C, k1=None, k2=None, alpha=1):
    """小規模問題の制約式を設定する関数"""
    if k2 is None:
        k2 = 2 * alpha / C + 1
    if k1 is None:
        k1 = (2 + 2 * alpha / C) + 1

    # 一人一台の車に乗車するという制約(1)
    allocate_constraints = [one_hot(q[i]) for i in range(npeople)]
    # 一台の車に乗れるのはC人以下だという制約(2)
    capacity_constraints = [less_equal(sum_poly(q[:, j]), C) for j in range(ncars)]

    constraints = k1 * sum(allocate_constraints) + k2 * sum(capacity_constraints)
    return constraints

In [ ]:
def construct(ncars, npeople, D, C, k1=None, k2=None, alpha=1):
    """分割後の小規模な問題のためのモデルを作成する関数"""
    D = regularizeDistance(D)
    gen = BinarySymbolGenerator()
    q = gen.array(npeople, ncars)

    cost = setObjective(q, ncars, npeople, D, C, alpha=alpha)
    constraints = setConstraints(q, ncars, npeople, C, k1=k1, k2=k2, alpha=alpha)
    model = cost + constraints
    return model, q

In [ ]:
def simple_plot(coord, ncars, title=None):
    m = folium.Map([sum(lat) / 2, sum(lon) / 2], tiles="Stamen Toner", zoom_start=zoom)
    tmp = list(coord.items())
    for j, x in enumerate(tmp):
        if j < ncars:
            folium.Marker(
                location=x[1][::-1],
                icon=folium.Icon(icon="car", prefix="fa", color=_colors[0]),
            ).add_to(m)
        else:
            folium.Marker(
                location=x[1][::-1],
                popup="person",
                icon=folium.Icon(icon="user", prefix="fa", color=_colors[1]),
            ).add_to(m)
    if title:
        title = f"<h4>{title}</h4>"
        m.get_root().html.add_child(folium.Element(title))
    return m

In [ ]:
def plot_result(coord, q_values, load_capacity, title=None):
    global _C
    m = folium.Map([sum(lat) / 2, sum(lon) / 2], tiles="Stamen Toner", zoom_start=zoom)
    legend = ""
    npeople = len(q_values)
    ncars = len(q_values[0])
    columns = ["latitude", "longitude", "size", "name"]
    data = {label: list() for label in columns}
    answer = dict()
    for i in range(npeople):
        car = np.where(np.array(q_values[i]) == 1)[0][-1]
        if car not in answer:
            answer[car] = []
        answer[car].append(i + ncars)

    for k in range(ncars):
        _color = _colors[k % len(_colors)]
        car_loc = coord[k]
        if k in answer:
            tmp = answer[k]
            x = [coord[p][0] for p in tmp] + [car_loc[0]]
            y = [coord[p][1] for p in tmp] + [car_loc[1]]
        else:
            x = car_loc[:1]
            y = car_loc[1:]
            _color = "gray"
        folium.Marker(
            location=[y[-1], x[-1]],
            popup=f"cluster{k}",
            icon=folium.Icon(icon="car", prefix="fa", color=_color),
        ).add_to(m)
        for a, b in zip(y[:-1], x[:-1]):
            folium.Marker(
                location=[a, b],
                popup=f"person{k}",
                icon=folium.Icon(
                    icon="user", prefix="fa", color="white", icon_color=_color
                ),
            ).add_to(m)
        legend += f"<font color={_color}> group{k}（{len(x) - 1}/{_C}）</font>"
        load_capacity[k] = len(x) - 1
    if title:
        title = f"<h4>{title}</h4>\n{legend}"
        m.get_root().html.add_child(folium.Element(title))
    display(m)

In [ ]:
def ride_share_initialize(
    alpha: float = 1.0,
    C: int = 1,
    npeople: int = 1,
    ncars: int = 1,
    timeout: int = 1000,
    _seed=0,
):
    global _ncars
    global _npeople
    global D
    global _C
    global coord
    global param
    global solver
    global load_capacity
    client = FixstarsClient()
    client.parameters.timeout = timeout
    solver = Solver(client)
    _ncars, _npeople, D, _C, coord = generate_problem(
        lon, lat, parking, ncars=ncars, C=C, npeople=npeople, seed=_seed
    )
    Parameter = namedtuple("Config", ("alpha"))
    param = Parameter(alpha=alpha)
    m = simple_plot(
        coord,
        ncars,
        title=f"Initital State（C={_C}, ncars={_ncars}, npeople={_npeople}）",
    )
    load_capacity = [0] * _ncars
    display(m)


def ride_share_solve(Progress):
    global _ncars
    global _npeople
    global D
    global _C
    global coord
    global param
    global solver
    global load_capacity
    model, q = construct(_ncars, _npeople, D, _C, alpha=param.alpha)
    Progress.value += 1
    result = solver.solve(model)
    Progress.value += 1
    if len(result) == 0:
        raise RuntimeError("No feasible solution was found.")
    energy, values = result[0].energy, result[0].values
    q_values = q.decode(values)
    Progress.value += 1
    num_used_cars = len(
        set([np.where(np.array(q_values)[i] == 1)[0][-1] for i in range(_npeople)])
    )
    title = "Result（active cars: {}/{}, energy: {:.3f}）".format(
        num_used_cars, _ncars, energy
    )
    Progress.value += 1
    plot_result(coord, q_values, load_capacity, title=title)
    Progress.value += 1


def ride_share_capacity():
    global _ncars
    global _C
    global load_capacity
    fig = plt.figure(figsize=(8, 6))
    plt.ylabel("number of people")
    plt.title("Capacity", y=1.08, size="x-large")
    hatchlist = ["", "///", "...", "---"]
    plt.bar(list(range(_ncars)), load_capacity)
    plt.grid(True, axis="y", color="gainsboro", alpha=0.8, zorder=7)
    plt.hlines(_C, -1, _ncars, color="red")
    plt.xticks(list(range(_ncars)), [f"car {k}" for k in range(_ncars)])
    plt.yticks([_C], [f"Capacity\n{_C}"], color="red")
    plt.show()

In [ ]:
alpha_slider = FloatSlider(
    value=1,
    min=0.1,
    max=5,
    step=0.1,
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="f",
)
C_slider = IntSlider(
    value=8,
    min=1,
    max=15,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
)
people_slider = IntSlider(
    value=20,
    min=0,
    max=50,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
)
car_slider = IntSlider(
    value=5,
    min=0,
    max=9,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
)
time_slider = IntSlider(
    value=1500,
    min=100,
    max=5000,
    step=50,
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
)

options1 = [Label(value="alpha :(大きいほど使用台数を減らすことを重視する)"), alpha_slider]
options1 += [Label(value="制限時間 [ ms ] :"), time_slider]

options2 = [Label(value="車の台数 ( ncars ) :"), car_slider]
options2 += [Label(value="利用者数 ( npeople ) :"), people_slider]
options3 = [Label(value="各駐車場あたりの人数の上限 ( C ) :"), C_slider]

options = [GridBox(options1), GridBox(options2), GridBox(options3)]
options = Accordion(children=[HBox(options)])
options.set_title(0, "Options")
options.selected_index = None

Progress = IntProgress(
    value=0,
    min=0,
    max=7,
    step=1,
    description="Solving...",
    bar_style="info",
    orientation="horizontal",
)
ride_share_run_btn = Button(
    description="Run", button_style="", tooltip="Run", icon="check"
)
ride_share_out = Output()
ride_share_capacity_out = Output()

ride_share_relocate_btn = Button(
    description="Change Location", button_style="", tooltip="Change Location", icon=""
)


def show_ride_share_problem(alpha, C, npeople, ncars, timeout, seed=0):
    global Progress
    Progress.value = 0
    ride_share_initialize(alpha, C, npeople, ncars, timeout, _seed=seed)
    with ride_share_capacity_out:
        ride_share_capacity_out.clear_output()
        ride_share_capacity()


def show_ride_share_problem_relocate(btn):
    global Progress
    Progress.value = 0
    with ride_share_capacity_out:
        ride_share_capacity_out.clear_output()
    with ride_share_out:
        ride_share_out.clear_output()
        ride_share_initialize(
            alpha_slider.value,
            C_slider.value,
            people_slider.value,
            car_slider.value,
            time_slider.value,
            _seed=None,
        )
    with ride_share_capacity_out:
        ride_share_capacity()


def show_ride_share_result(btn):
    global Progress
    Progress.value = 0
    with ride_share_capacity_out:
        ride_share_capacity_out.clear_output()
    with ride_share_out:
        Progress.value += 1
        ride_share_out.clear_output()
        Progress.value += 1
        ride_share_solve(Progress)
    with ride_share_capacity_out:
        ride_share_capacity()


ride_share_out = interactive_output(
    show_ride_share_problem,
    {
        "alpha": alpha_slider,
        "C": C_slider,
        "npeople": people_slider,
        "ncars": car_slider,
        "timeout": time_slider,
    },
)
ride_share_relocate_btn.on_click(show_ride_share_problem_relocate)
ride_share_run_btn.on_click(show_ride_share_result)

grid = GridspecLayout(10, 10)
right = 3

# fill it in with widgets
grid[1:, :right] = ride_share_out
grid[0, 0] = ride_share_run_btn
grid[0, 1] = ride_share_relocate_btn
grid[0, 2] = Progress
grid[1:, right:] = ride_share_capacity_out


display(VBox([options, grid]))